In [252]:
import numpy as np
from scipy.stats import norm
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions
tfb = tfp.bijectors

In [12]:
from tensorflow_probability.python.vi import csiszar_divergence

In [13]:
csiszar_divergence

<module 'tensorflow_probability.python.vi.csiszar_divergence' from '/usr/local/lib/python3.8/site-packages/tensorflow_probability/python/vi/csiszar_divergence.py'>

In [15]:
import tensorflow_probability as tfp

In [16]:
tfp.vi.kl_reverse

<function tensorflow_probability.python.vi.csiszar_divergence.kl_reverse(logu, self_normalized=False, name=None)>

In [17]:
csiszar_divergence.kl_reverse

<function tensorflow_probability.python.vi.csiszar_divergence.kl_reverse(logu, self_normalized=False, name=None)>

In [18]:
import pymc4 as pm

In [20]:
import numpy as np

@pm.model
def model():
    mu = yield pm.Normal('mu', 0, 10)
    sigma = yield pm.Exponential('sigma', 1)
    ll = yield pm.Normal('ll', mu, sigma, observed=np.random.normal(12, 2.2, 200))

In [21]:
a = pm.MeanField(model())

In [24]:
a.approx.reparameterization_type

<Reparameteriation Type: FULLY_REPARAMETERIZED>

In [26]:
tfd = tfp.distributions
mu = [1., 2, 3]
cov = [[ 0.36,  0.12,  0.06],
       [ 0.12,  0.29, -0.13],
       [ 0.06, -0.13,  0.26]]
scale = tf.linalg.cholesky(cov)

mvn = tfd.MultivariateNormalTriL(
    loc=mu,
    scale_tril=scale)

In [109]:
def monte_carlo_expection(function, samples):
    return tf.reduce_mean(function(samples), axis=0)

In [104]:
f = lambda u: u
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    p = tfd.Normal(loc=x, scale=1.)
    efx_true = x
    samples = p.sample(int(1e5), seed=1)
    mean = tf.reduce_mean(p.sample(1e5), axis=0)
    efx_reparam = tfp.monte_carlo.expectation(f, samples)

In [ ]:
q_sample = q.sample()
kl = q.log_prob(q_sample) - p.log_prob(q_sample)
tf.reduce_mean(kl, axis=0)

In [105]:
tape.gradient(efx_true, x)

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [106]:
tape.gradient(efx_reparam, x)

<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([1.0009902, 1.0009902, 1.0009902, 1.0009902, 1.0009902, 1.0009902,
       1.0009902, 1.0009902], dtype=float32)>

In [107]:
tape.gradient(mean, x)

<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([1.0009902, 1.0009902, 1.0009902, 1.0009902, 1.0009902, 1.0009902,
       1.0009902, 1.0009902], dtype=float32)>

In [102]:
tape.gradient(tf.reduce_mean(p.sample(1e5), axis=0), x)

In [103]:
tf.reduce_mean(p.sample(1e5), axis=0)

<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([-9.9955738e+05, -9.9996002e+01, -1.0004002e+01, -9.9946147e-01,
        1.0012723e+00,  9.9980774e+00,  1.0000111e+02,  9.9955731e+05],
      dtype=float32)>

In [108]:
tf.reduce_mean

<function tensorflow.python.ops.math_ops.reduce_mean(input_tensor, axis=None, keepdims=False, name=None)>

In [110]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x**2

In [113]:
tape.watched_variables()
tape.gradient(y, x)

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [164]:
unknown_mean = -5
known_sigma = 3
data_points = 1000
data = np.random.normal(unknown_mean, known_sigma, size=data_points)
prior_mean = 4
prior_sigma = 2

# References - http://patricklam.org/teaching/conjugacy_print.pdf
precision = 1 / prior_sigma ** 2 + data_points / known_sigma ** 2
estimated_mean = (
    prior_mean / prior_sigma ** 2 + (data_points * np.mean(data) / known_sigma ** 2)
) / precision

@pm.model
def model():
    mu = yield pm.Normal("mu", prior_mean, prior_sigma)
    ll = yield pm.Normal("ll", mu, known_sigma, observed=data)

In [174]:
def model(x):
    mu_prior = tf.math.exp(-(((x - prior_mean)/prior_sigma)**2)/2)/(np.sqrt(2*np.pi*(prior_sigma**2)))
    ll = mu_prior = tf.math.exp(-(((data - mu_prior)/known_sigma)**2)/2)/(np.sqrt(2*np.pi*(known_sigma**2)))
    return sum(map(tf.reduce_sum, [
        tf.math.log(mu_prior),
        tf.math.log(ll)
    ]))

In [234]:
from scipy.stats import norm

def model(x):
    mu_prior = norm(prior_mean, prior_sigma)
    ll = norm(mu_prior.rvs(), known_sigma)
    return sum(map(tf.reduce_sum, [
        mu_prior.logpdf(x),
        ll.logpdf(data)
    ]))
    

In [235]:
joint_log_prob = lambda *samples: model(*samples)

In [268]:
mu_var = tf.Variable(tf.random.normal([]))
cov_param = tfp.util.TransformedVariable(tf.ones([]), tfb.Softplus())

In [270]:
cov_param

<TransformedVariable: dtype=float32, shape=[], fn="softplus", numpy=1.0>

In [267]:
a, b = tf.constant([1., 2.]), tf.constant(2.)
tf.random.normal(mu_var, cov_param)

InvalidArgumentError: Value for attr 'T' of float is not in the list of allowed values: int32, int64
	; NodeDef: {{node RandomStandardNormal}}; Op<name=RandomStandardNormal; signature=shape:T -> output:dtype; attr=seed:int,default=0; attr=seed2:int,default=0; attr=dtype:type,allowed=[DT_HALF, DT_BFLOAT16, DT_FLOAT, DT_DOUBLE]; attr=T:type,allowed=[DT_INT32, DT_INT64]; is_stateful=true> [Op:RandomStandardNormal]

In [265]:
with tf.GradientTape() as tape:
    tape.watch(mu_var)



In [266]:
tape.watched_variables()

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.42901072>,)

In [247]:
cov_param = tfp.util.TransformedVariable(
            tf.ones(flattened_shape, dtype=dtype), tfb.Softplus(), name="sigma"
        )

NameError: name 'flattened_shape' is not defined